In [ ]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

# Load the trained model
def load_model():
    # Load data from data15.csv
    data = pd.read_csv('data.csv')

    # Define input and output features
    X = data.iloc[:, :9]  # Input features
    y = data.iloc[:, 9]   # Output (Relative Density)

    # Standardize the input features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Initialize the GradientBoostingRegressor with best parameters
    model = GradientBoostingRegressor(
          learning_rate=0.1309986281844441,
            n_estimators=69,
            max_depth=7,
            min_samples_split=6,
            min_samples_leaf=1,
            random_state=42
    )

    # Fit the model
    model.fit(X_scaled, y)
    return model, scaler

# Predict the output using the loaded model
def predict_density():
    try:
        machine_values = [machine_entries[i].get() for i in range(len(machine_properties))]
        material_values = [material_entries[i].get() for i in range(len(material_properties))]

        machine_dict = {machine_properties[i]: float(value) for i, value in enumerate(machine_values)}
        material_dict = {material_properties[i]: float(value) for i, value in enumerate(material_values)}

        machine_ordered = [machine_dict[feature] for feature in machine_properties]
        material_ordered = [material_dict[feature] for feature in material_properties]

        features_ordered = machine_ordered + material_ordered
        
        features_scaled = scaler.transform([features_ordered])
        predicted_density = model.predict(features_scaled)[0]
        result_label.config(text=f"Predicted Relative Density: {predicted_density:.2f}")
    except ValueError:
        messagebox.showerror("Error", "Please enter valid numerical values")

# GUI setup
window = tk.Tk()
window.title("Relative Density Prediction For Laser Powder Bed Fusion")
window.geometry("800x350")  # Set window size

model, scaler = load_model()

machine_properties = ["Power (W)", "Scanning speed (mm/s)", "Hatch spacing (mm)", "Layer thickness (mm)", "Energy density (J/mm^3)",
                      "Spot diameter (mm)"]
material_properties = ["Thermal conductivity (W/m-K)", "Specific heat (J/Kg-K)", "Dimensionless_number"]

machine_frame = tk.LabelFrame(window, text="Machine Parameters")
machine_frame.pack(side=tk.LEFT, padx=10, pady=10)

material_frame = tk.LabelFrame(window, text="Material Parameters")
material_frame.pack(side=tk.RIGHT, padx=10, pady=10)

machine_entries = []
material_entries = []

for i, feature in enumerate(machine_properties):
    tk.Label(machine_frame, text=f"{feature}: ").grid(row=i, column=0)
    entry = tk.Entry(machine_frame)
    entry.grid(row=i, column=1)
    machine_entries.append(entry)

for i, feature in enumerate(material_properties):
    tk.Label(material_frame, text=f"{feature}: ").grid(row=i, column=0)
    entry = tk.Entry(material_frame)
    entry.grid(row=i, column=1)
    material_entries.append(entry)

predict_button = tk.Button(window, text="Predict", command=predict_density)
predict_button.pack(pady=10)

result_label = tk.Label(window, text="")
result_label.pack()

window.mainloop()
